# TouchTerrain standalone in a jupyter notebook
Chris Harding, June 15, 2020 (<charding@iastate.edu>)

- this jupyter notebook runs a standalone version of TouchTerrain, similar to `TouchTerrain_standalone.py`
- this notebook needs to be run in Python __3.x__  (I'm using 3.7)
- Use pip to build and install a package called touchterrain (all lowercase!), which contains all functions needed to run in standalone and in server mode. This will also install all dependencies needed(!)


- you can run pip either in a terminal or inside a jupyer cell (next cell)
- to run it in a separate terminal, `cd` to the folder that contains the `setup.py` file (which will contain a folder called touchterrain)
    - enter:  `pip install .` 
    - note the . at the end, which will make pip run the script in setup.py
    - this will install all dependencies (including earthengine-api) 
- on Windows, if pip gives you trouble with __gdal__, get the whl file from [here](https://www.lfd.uci.edu/~gohlke/pythonlibs/) and install it separatly with `pip install <whl file>` then go back and try the full install again
- on Mac, gdal should be avaiable via [conda-forge](https://anaconda.org/conda-forge/gdal), however I have run into trouble with shared library mismatches which I could not resolve! 




In [ ]:
# if you have already installed touchterrain, skip this cell!

# Run this cell (Shift-Enter) to have pip install the touchterrain module and all its dependencies
# Warning: this can take a awhile! You may not see any message during the install but you should see the full log when pip is done (or when you got errors)
!pip3 install .

In [ ]:
# import packages
import os, sys
from pprint import pprint

# earthengine-api and touchterrain should have been installed via pip earlier
import ee
from touchterrain.common import TouchTerrainEarthEngine as TouchTerrain

# if you get errors running this cell, install the requested packages

## If you're not going to use Earth Engine's (EE) online DEM data skip the next section and go to _Running Touchterrain standalone_


### Preparing to use Google Earth Engine online DEM rasters 

- You don't need this if you only want to import terrain from locally stored raster files (geotiffs)!
- TouchTerrain can use DEM data from Google Earth Engine (given the corners of the area), but you need to first request a developer account and set up an authentication file
- (This dev account is different from your standard Google (Gmail, etc.) account!)
- Getting the account is free and entitles you to a modest number of requests (4 per seconds), which are also free. To request got to https://signup.earthengine.google.com/, you'll get and email with a file. 
- refer to the part __Setting Up Authentication Credentials__ https://developers.google.com/earth-engine/python_install (ignore the stuff above it, as you should already have installed all the needed packages when you installed the earthengine-api package ...)


- we have already imported ee but we will need to run `ee.Initialize()` before using the Earth Engine API. `ee.Initialize()` will try to access a local authentication file, which `ee.Authenticate()` will create
- To run `ee.Authenticate()`, comment in and run the next cell, which will end up creating a authentication file for your local system. After you have done this ONCE, you should not have to use `ee.Authenticate()`, so you can comment it out again.


In [ ]:
#ee.Authenticate()

- When you use: `ee.Authenticate()` this text should appear and you should be redirected to a webpage:
```
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does  not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=5172225062...

```
- the web page will have you select a Google account for use with ee and give you a code
- paste in the code: `Enter verification code: <your code>` and hit Enter, you should get `Successfully saved authorization token.`
- this will create a folder `.config` in your home folder and create an authentication file (in a earthengine folder) that `ee.Initialize()` will to authenticate you. 
- You need to run `ee.Initialize()` at least once before you can use online data from Earth Engine

In [ ]:
# skip this cell (or comment it out) if you do NOT want to use Earth Engine DEM data 
# but instead only want to use a local geotiff as DEM
ee.Initialize()

## Running Touchterrain standalone (local and online DEM data)

- Put your settings into the dictionary below and hit Shift-Enter


- for more info on the settings, look at the ReadMe on https://github.com/ChHarding/TouchTerrain_for_CAGEO
- note, however, that the settings given below are in Python syntax, whereas the ReadMe describes the JSON syntax used in the config file
- both are very similar except for None and True/False

``` 
    Python:  JSON:
    None     null
    True     true
    False    false
```



In [ ]:
args = {
    # DEM/Area to print
    
    # A: use local DEM raster (geotiff)
    #"importedDEM": "stuff/pyramid.tif",  # path to the geotif in relation to where this notebook sits
    
    # B: use area and a DEM online source via EarthEngine
    "importedDEM": None,
    "DEM_name": "USGS/NED",   # DEM source
    "bllat": 44.50185267072875,   # bottom left corner lat
    "bllon": -108.25427910156247, # bottom left corner long
    "trlat": 44.69741706507476,   # top right corner lat
    "trlon": -107.97962089843747, # top right corner long

    # 3D print parameters
    "tilewidth": 200,  # width of each tile in mm, (tile height will be auto calculated)
    "printres": 0.4,  # resolution (horizontal) of 3D printer in mm, should be your NOZZLE size or just a bit less! 
                      # Using something like 0.01 will NOT print out a super detailed version as you slicer will remove
                      # super fine details anyway! You'll just wait a long time and get a super large STL file!
    
                      # If you want the equivalent of the original resolution of the DEM, use -1 (But again, think of the slicer ...)
                      # This will work for any local DEM but only for small online DEMs b/c 
                      # Google now (Mar. 2020) caps raster downloads at 10 Mega pixels!
    
    "ntilesx": 1, # number of tiles in x  
    "ntilesy": 1, # number of tiles in y    

    "basethick": 0.5,   # thickness (in mm) of printed base
    "zscale": 1,      # elevation (vertical) scaling
    "fileformat": "STLb",  # format of 3D model files: "obj" wavefront obj (ascii),
                           #   "STLa" ascii STL or "STLb" binary STL.
                           #   To export just the (untiled) raster (no mesh), use "GeoTiff" 
    "zip_file_name": "myterrain",   # base name of zipfile, .zip will be added

    
    # Expert settings
    "tile_centered": False, # True-> all tiles are centered around 0/0, False, all tiles "fit together"
    "CPU_cores_to_use" : 0, # 0: use all available cores, None: don't use multiprocessing (single core only)
                            # multi-core will be much faster for more than 1 tile 
    "max_cells_for_memory_only" : 5000^2, # if number of raster cells is bigger than this, use temp_files instead of memory.
                            # set this very high to force use of memory and lower it if you run out of memory
    "no_bottom": False,   # omit bottom triangles? Most slicers still work and it makes smaller files
    "no_normal": True,    # Don't calculate normals for triangles. This is significantly faster but some viewer may need them.
    "bottom_image": None, # 1 band greyscale image used for bottom relief
    "ignore_leq": None,   # set all values <= this to NaN so they don't print
    "lower_leq": None,    # e.g. [0.0, 2.0] values <= 0.0 will be lowered by 2mm in the final model
    "unprojected": False, # don't project to UTM (for EE rasters only)
    "projection": None,   # None means use the closest UTM zone. Can be a EPSG number (int!) instead but not all work. 
    "only" : None,        # if not None: list with x and y tile index (1 based) of the only tile to process
                          #   e.g. [1,1] will only process the tile in upper left corner, [2,1] the tile right to it, etc.
    "importedGPX": [],    # 
}
########################################################

# if we want to work on a local raster, get the full pathname to it
if args["importedDEM"] != None: 
    from os.path import abspath
    args["importedDEM"]= abspath(args["importedDEM"]) 
    print("reading in local DEM:", args["importedDEM"])
print("settings stored, ready to process")

In [ ]:
# Optional: drape GPX path from files over the terrain (thanks to KohlhardtC!)
# below is an area covered by a bunch of test GPX files in the gpx-test folder.
# Note: you must uncomment the last line in this cell to actually use these gpx settings!
from touchterrain.common.TouchTerrainGPX import *
gpx_args = {   
    # Coordinates relevant to GPX test files
    "bllat": 39.32205105794382,   # bottom left corner lat
    "bllon": -120.37497608519418, # bottom left corner long
    "trlat": 39.45763749030933,   # top right corner lat
    "trlon": -120.2002248034559, # top right corner long

    "importedGPX": # Plot GPX paths from these files onto the model.
                 ["stuff/gpx-test/DLRTnML.gpx",
                  "stuff/gpx-test/DonnerToFrog.gpx",
                  "stuff/gpx-test/CinTwistToFrog.gpx",
                  "stuff/gpx-test/sagehen.gpx",
                  "stuff/gpx-test/dd-to-prosser.gpx",
                  "stuff/gpx-test/alder-creek-to-crabtree-canyon.gpx",
                  "stuff/gpx-test/ugly-pop-without-solvang.gpx",
                  "stuff/gpx-test/tomstrail.gpx"   
                 ],
     "gpxPathHeight": 40,  # Currently we plot the GPX path by simply adjusting the raster elevation at the specified lat/lon,
                           # therefore this is in meters. Negative numbers are ok and put a dent in the mdoel  
     "gpxPixelsBetweenPoints" : 20, # GPX Files haves a lot of points. A higher number will create more space between lines drawn
                                    # on the model and can have the effect of making the paths look a bit cleaner 
     "gpxPathThickness" : 5, # Stack paralell lines on either side of primary line to create thickness. 
                             # A setting of 1 probably looks the best
}
#args = {**args, **gpx_args}; print(args) # merge gpx_args into args, ** unrolls dicts

In [ ]:
#
# Process the data
#

# This may take some time! You'll see In[*] and some log messages (will also be in the logfile inside the zip)
# You may see some red stuff with 10%, etc. - don't worry, that's normal
totalsize, full_zip_file_name = TouchTerrain.get_zipped_tiles(**args) # all args are in a dict
print("\nDONE!\n\nCreated zip file", full_zip_file_name,  "%.2f" % totalsize, "Mb")

# your zip file will be inside the tmp folder which is inside the same folder your notebook file is in

In [ ]:
# If you want to unzip the zip file, run this cell
# (You will need to do this before using k3d for visualization)

import os.path
from glob import glob
folder, file = os.path.splitext(full_zip_file_name) # get folder of zip file

# unzip the zipfile into the folder it's already in
import zipfile
zip_ref = zipfile.ZipFile(full_zip_file_name, 'r')
zip_ref.extractall(folder)
zip_ref.close()
print ("unzipped files from", full_zip_file_name, "into the folder", folder)
print (folder, "contains these files:")
for f in glob(folder + os.sep + "*.*"): print(" ", f)

## Visualize the STL file(s)

- If you want to visualize your model(s), install k3d (`pip install k3d`) and run the cell below
- this won't work with OBJ files, as k3d can't read them in.
- binary STL should work, but ascii may not.


In [ ]:
import k3d

# get all stl files in that folder
mesh_files = glob(folder + os.sep + "*.stl")
#print "in folder", folder, "using", mesh_files

plot = k3d.plot()

from random import randint
for m in mesh_files:
    col = (randint(0,255) << 16) + (randint(0,255) << 8) + randint(0,255) # random rgb color as hex
    print("adding to viewer:", m, hex(col))
    buf = open(m, 'rb').read()
    plot += k3d.stl(buf, color=col)
plot.display()